In [ ]:
!pip uninstall -y keras
!pip install 'keras<3.0.0' mediapipe-model-maker


Found existing installation: keras 3.9.1
Uninstalling keras-3.9.1:
  Successfully uninstalled keras-3.9.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
import pandas as pd
data = pd.read_csv('online_payement_fraud_detection.csv')

# Assuming 'data' is your dataframe and 'isFraud' column is what you're using for classification
fraud = data[data['isFraud'] == 1]
non_fraud = data[data['isFraud'] == 0]

# Sample 10,000 non-fraud cases (randomly sampled)
non_fraud_sampled = non_fraud.sample(n=500, random_state=42)

# Get all 500 fraud cases
fraud_sampled = fraud.sample(n=500, random_state=42)

# Combine the two datasets
balanced_data = pd.concat([non_fraud_sampled, fraud_sampled])

# Shuffle the data to mix fraud and non-fraud entries
df = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)


# ✅ Drop rows with missing target
df = df.dropna()

# ✅ Separate features and target
X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ✅ Encode categorical features
cat_cols = X.select_dtypes(include=['object']).columns
ct = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)],
    remainder="passthrough"
)
X = ct.fit_transform(X)

# ✅ Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ Apply SMOTE
minority_class_size = y_train.value_counts().min()
k = min(5, minority_class_size - 1)

sm = SMOTE(random_state=42, k_neighbors=k)
X_train, y_train = sm.fit_resample(X_train, y_train)

# ✅ Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model.predict(X_test)
y_pred = (y_prob > 0.5).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 53ms/step - loss: 2.5128 - precision_10: 0.5491 - recall_10: 0.4322 - val_loss: 1.4006 - val_precision_10: 0.6689 - val_recall_10: 0.9900
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.0503 - precision_10: 0.9267 - recall_10: 0.9662 - val_loss: 1.3230 - val_precision_10: 0.6689 - val_recall_10: 0.9900
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.6444 - precision_10: 0.9855 - recall_10: 0.9991 - val_loss: 2.3533 - val_precision_10: 1.0000 - val_recall_10: 0.0300
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.5499 - precision_10: 0.9821 - recall_10: 0.9546 - val_loss: 1.1813 - val_precision_10: 0.6735 - val_recall_10: 0.9900
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.7567 - precision_10: 0.9784 - recall_10: 0.9688 - val_loss: 1.6953 - val_precision_10: 0.6711 - val_recall_10: 1.0000
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.9213 - precision_10: 0.9537 - recall_10: 0.9

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
import pandas as pd
data = pd.read_csv('online_payement_fraud_detection.csv')

# Assuming 'data' is your dataframe and 'isFraud' column is what you're using for classification
fraud = data[data['isFraud'] == 1]
non_fraud = data[data['isFraud'] == 0]

# Sample 10,000 non-fraud cases (randomly sampled)
non_fraud_sampled = non_fraud.sample(n=500, random_state=42)

# Get all 500 fraud cases
fraud_sampled = fraud.sample(n=500, random_state=42)

# Combine the two datasets
balanced_data = pd.concat([non_fraud_sampled, fraud_sampled])

# Shuffle the data to mix fraud and non-fraud entries
df = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)


# ✅ Drop rows with missing target
df = df.dropna()

# ✅ Separate features and target
X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ✅ Encode categorical features
cat_cols = X.select_dtypes(include=['object']).columns
ct = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)],
    remainder="passthrough"
)
X = ct.fit_transform(X)

# ✅ Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ Apply SMOTE
minority_class_size = y_train.value_counts().min()
k = min(5, minority_class_size - 1)

sm = SMOTE(random_state=42, k_neighbors=k)
X_train, y_train = sm.fit_resample(X_train, y_train)

# ✅ Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model.predict(X_test)
y_pred = (y_prob > 0.5).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 2.7299 - precision_21: 0.4831 - recall_21: 0.6865 - val_loss: 1.5854 - val_precision_21: 0.7029 - val_recall_21: 0.9700
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 1.2496 - precision_21: 0.8929 - recall_21: 0.9274 - val_loss: 1.1517 - val_precision_21: 0.7561 - val_recall_21: 0.9300
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.7651 - precision_21: 0.9931 - recall_21: 0.9915 - val_loss: 0.8633 - val_precision_21: 0.7586 - val_recall_21: 0.8800
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.4432 - precision_21: 0.9979 - recall_21: 0.9971 - val_loss: 0.9251 - val_precision_21: 0.6993 - val_recall_21: 1.0000
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.3121 - precision_21: 0.9916 - recall_21: 0.9906 - val_loss: 2.1396 - val_precision_21: 0.5102 - val_recall_21: 1.0000
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.4234 - precision_21: 0.9772 - recall_21: 0.9857 - val_l

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# ✅ Load dataset
import pandas as pd
data = pd.read_csv('online_payement_fraud_detection.csv',on_bad_lines='skip')

# Assuming 'data' is your dataframe and 'isFraud' column is what you're using for classification
fraud = data[data['isFraud'] == 1]
non_fraud = data[data['isFraud'] == 0]
nu = list(data['isFraud'].value_counts())[1]
# Sample 10,000 non-fraud cases (randomly sampled)
non_fraud_sampled = non_fraud.sample(n=nu, random_state=42)

# Get all 500 fraud cases
fraud_sampled = fraud.sample(n=nu, random_state=42)

# Combine the two datasets
balanced_data = pd.concat([non_fraud_sampled, fraud_sampled])

# Shuffle the data to mix fraud and non-fraud entries
df = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)


# ✅ Drop rows with missing target
df = df.dropna()

# ✅ Separate features and target
X = df.drop(columns=['isFraud'])
y = df['isFraud']

# ✅ Encode categorical features
cat_cols = X.select_dtypes(include=['object']).columns
ct = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)],
    remainder="passthrough"
)
X = ct.fit_transform(X)

# ✅ Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ Apply SMOTE
minority_class_size = y_train.value_counts().min()
k = min(5, minority_class_size - 1)

sm = SMOTE(random_state=42, k_neighbors=k)
X_train, y_train = sm.fit_resample(X_train, y_train)

# ✅ Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Build model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model.predict(X_test)
y_pred = (y_prob > 0.5).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 2.6870 - precision: 0.6667 - recall: 0.5679 - val_loss: 5.7162 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - loss: 3.0772 - precision: 0.8779 - recall: 0.8357 - val_loss: 1.5938 - val_precision: 0.7259 - val_recall: 0.9333
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 1.3851 - precision: 0.9535 - recall: 0.9568 - val_loss: 6.6181 - val_precision: 0.5006 - val_recall: 1.0000
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 2.8990 - precision: 0.9192 - recall: 0.9456 - val_loss: 6.1947 - val_precision: 0.5000 - val_recall: 0.0024
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 3.5217 - precision: 0.9500 - recall: 0.9153 - val_loss: 5.2258 - val_precision: 0.5036 - val_recall: 1.0000
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.2850 - precision: 0.9351 - recall: 0.9422 - val_loss: 3.7715 - val_precision: 0.6931 - val_recal

In [ ]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model_2.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model_2.predict(X_test)
y_pred = (y_prob > 0.5).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - loss: 2.4401 - precision_1: 0.6291 - recall_1: 0.5159 - val_loss: 6.0176 - val_precision_1: 0.9615 - val_recall_1: 0.0595
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - loss: 1.9886 - precision_1: 0.9029 - recall_1: 0.9261 - val_loss: 2.6232 - val_precision_1: 0.8207 - val_recall_1: 0.7738
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 31ms/step - loss: 2.1342 - precision_1: 0.9669 - recall_1: 0.9761 - val_loss: 2.9770 - val_precision_1: 0.5850 - val_recall_1: 1.0000
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 2.3010 - precision_1: 0.9581 - recall_1: 0.9702 - val_loss: 5.2666 - val_precision_1: 0.9583 - val_recall_1: 0.0548
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 2.8682 - precision_1: 0.9588 - recall_1: 0.9461 - val_loss: 4.6750 - val_precision_1: 0.6259 - val_recall_1: 1.0000
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.9931 - precision_1: 0.9600 - recall_1: 0.9690 - val_loss: 3.7508

In [ ]:
model_2_1 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_2_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model_2_1.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model_2_1.predict(X_test)
y_pred = (y_prob > 0.5).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 2.6514 - precision_13: 0.6334 - recall_13: 0.6574 - val_loss: 7.4934 - val_precision_13: 1.0000 - val_recall_13: 0.0238
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 2.0603 - precision_13: 0.9344 - recall_13: 0.9399 - val_loss: 5.4212 - val_precision_13: 0.5012 - val_recall_13: 1.0000
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 2.2669 - precision_13: 0.9281 - recall_13: 0.9430 - val_loss: 4.0279 - val_precision_13: 0.9714 - val_recall_13: 0.0810
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 2.6796 - precision_13: 0.9457 - recall_13: 0.9515 - val_loss: 5.8804 - val_precision_13: 1.0000 - val_recall_13: 0.0095
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 3.3090 - precision_13: 0.9425 - recall_13: 0.9327 - val_loss: 4.6535 - val_precision_13: 0.9630 - val_recall_13: 0.0619
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4745 - precision_13: 0.9348 - recall_13: 0.9

In [ ]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model_3.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model_3.predict(X_test)
y_pred = (y_prob > 0.5).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 37ms/step - loss: 2.8085 - precision_2: 0.5954 - recall_2: 0.6864 - val_loss: 21.7340 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 2.7011 - precision_2: 0.8995 - recall_2: 0.8693 - val_loss: 2.4470 - val_precision_2: 0.6990 - val_recall_2: 0.9952
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1.6974 - precision_2: 0.9648 - recall_2: 0.9768 - val_loss: 4.1932 - val_precision_2: 0.5066 - val_recall_2: 1.0000
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 2.2804 - precision_2: 0.9527 - recall_2: 0.9671 - val_loss: 7.3536 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 3.1326 - precision_2: 0.9591 - recall_2: 0.9266 - val_loss: 6.5342 - val_precision_2: 0.5006 - val_recall_2: 1.0000
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 3.1828 - precision_2: 0.9363 - recall_2: 0.9749 -

In [ ]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model_3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.009),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# ✅ Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# ✅ Train
history = model_3.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

# ✅ Predict with custom threshold
y_prob = model_3.predict(X_test)
y_pred = (y_prob > 0.7).astype(int)

# ✅ Classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 9s 99ms/step - loss: 6.3017 - precision_9: 0.5581 - recall_9: 0.6194 - val_loss: 5.3955 - val_precision_9: 0.5006 - val_recall_9: 1.0000
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - loss: 4.0395 - precision_9: 0.8460 - recall_9: 0.9187 - val_loss: 5.3142 - val_precision_9: 0.6919 - val_recall_9: 1.0000
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 4.6382 - precision_9: 0.9457 - recall_9: 0.9625 - val_loss: 6.2865 - val_precision_9: 0.5006 - val_recall_9: 1.0000
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 6s 94ms/step - loss: 4.5319 - precision_9: 0.9311 - recall_9: 0.9540 - val_loss: 6.7827 - val_precision_9: 0.5006 - val_recall_9: 1.0000
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - loss: 5.4345 - precision_9: 0.9124 - recall_9: 0.9634 - val_loss: 6.3842 - val_precision_9: 0.6908 - val_recall_9: 1.0000
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 82ms/step - loss: 5.9031 - precision_9: 0.9330 - recall_9: 0.9591 - val_loss: 7.3309